In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
import pandas as pd
%matplotlib inline 

###### VISÃO PROJEÇÃO TRANSACIONAL PRODUÇÃO

dataset13 = pd.read_csv("../input/sep-dataset-37/20161231_20190806_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset14 = pd.read_csv("../input/sep-dataset-38/20161231_20190813_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset15 = pd.read_csv("../input/sep-dataset-39/20161231_20190818_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset16 = pd.read_csv("../input/sep-dataset-40/20161231_20190822_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset17 = pd.read_csv("../input/sep-dataset-41/20161231_20190901_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset18 = pd.read_csv("../input/sep-dataset-42/20161231_20190903_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset19 = pd.read_csv("../input/sep-dataset-43/20161231_20190905_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset20 = pd.read_csv("../input/sep-dataset-44/20161231_20190908_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset21 = pd.read_csv("../input/sep-dataset-45/20161231_20190909_v1.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset22 = pd.read_csv("../input/sep-dataset-46/20161231_20190916.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset23 = pd.read_csv("../input/sep-dataset-47/20161231_20190923.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset24 = pd.read_csv("../input/sep-dataset-48/20161231_20190930csv.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset25 = pd.read_csv("../input/sep-dataset-49/20161231_20191008.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset26 = pd.read_csv("../input/sep-dataset-50/20161231_20191011.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset27 = pd.read_csv("../input/sep-dataset-51/20161231_20191015.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])

dataset = dataset25.copy()

daily_df = dataset[['DATA','TRN_TOTAL']][dataset['FL_ATIVO'] == 1]
daily_df.head()
daily_df.tail(12).sum()
#daily_df = daily_df[daily_df['DATA'] <= '2019-04-08']
#daily_df = daily_df[daily_df['DATA'] >= '2016-12-31']
daily_df = daily_df.groupby('DATA')['TRN_TOTAL'].sum()
daily_df = daily_df.resample('D').apply(sum)
df = daily_df.reset_index()
df.columns = ['ds', 'y']
df.head()
df.tail()


hldys1 = pd.DataFrame({
  'holiday': 'queda',
  'ds': pd.to_datetime([
                        '2017-02-28', '2017-04-14', '2017-04-16', '2017-04-21', '2017-05-01', '2017-06-15', 
                        '2017-09-07', '2017-10-12', '2017-11-02', '2017-11-15',
                        '2018-02-12', '2018-02-13', '2018-03-30', '2018-04-01', '2018-04-21', '2018-05-01', '2018-05-31',
                        '2018-09-07', '2018-10-12', '2018-11-02', '2018-11-15', '2019-03-04', '2019-03-05','2019-04-19',
                        '2019-04-21', '2019-05-01', '2019-06-20', '2019-09-07', '2019-10-12','2019-11-02','2019-11-15']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys2 = pd.DataFrame({
  'holiday': 'alta_sep',
  'ds': pd.to_datetime(['2017-12-20','2018-12-20','2019-12-20']),
  'lower_window': 0,
  'upper_window': 2,
})

hldys3 = pd.DataFrame({
  'holiday': 'especial',
  'ds': pd.to_datetime(['2017-12-25','2018-01-01','2018-12-25','2019-01-01','2019-12-25']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys4 = pd.DataFrame({
  'holiday': 'alta',
  'ds': pd.to_datetime(['2017-02-24','2017-03-01','2017-03-02', '2017-06-10', '2017-08-12', '2017-11-24', #'2017-05-13',
                        '2018-02-09','2018-02-14','2018-02-15', '2018-06-12', '2018-08-11', '2018-11-23',#'2018-05-12',
                        '2019-03-01','2019-03-06','2019-03-07','2019-05-11','2019-06-12','2019-08-10','2019-11-22']),
  'lower_window': 0,
  'upper_window': 0,
})
#'2017-02-24','2017-03-01','2017-03-02',
#'2018-02-09','2018-02-14','2018-02-15',
#'2019-03-01','2019-03-06','2019-03-07',
hldys = pd.concat((hldys1, hldys2, hldys3, hldys4))


prediction_size = 160
train_df = df.copy()
#train_df = df[:-prediction_size]
train_df['y'] = np.log(train_df['y'])
train_df.tail(n=6)

,ds,y
1006,2019-10-03,11.920209
1007,2019-10-04,12.218149
1008,2019-10-05,11.953075
1009,2019-10-06,11.195911
1010,2019-10-07,12.210981
1011,2019-10-08,12.075389


In [2]:
## modelo otimizado para rodar intra mês, isto é, ser preciso a partir do dia 08 a 15
j = 31 #16  # weekly seasonality prior_scale
i = 46 #31  # monthly seasonality prior_scale
b = 31 #16  # seasonality prior_scale
c = 46 #50  # holidays seasonality prior_scale
m = Prophet(yearly_seasonality=False,weekly_seasonality=False,growth = 'linear',
            daily_seasonality=False, holidays = hldys, holidays_prior_scale = c,
           seasonality_prior_scale = b, seasonality_mode = 'additive',
           changepoint_prior_scale = 0.99 , changepoint_range = 1,
           ).add_seasonality(
            name='w',period=7,fourier_order=50, prior_scale = 2*j).add_seasonality(
            name='m',period=30.5,fourier_order=20, prior_scale = i).add_seasonality(
            name='ano',period=365,fourier_order=40, prior_scale = 1.3*i)
m.fit(train_df)


future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

forecast = m.predict(future)
a = forecast.set_index('ds')[['yhat']].join(df.set_index('ds'))
a['yhat'] = np.exp(a['yhat'])

In [3]:
b = dataset.set_index('DATA')['2019-10-01':'2019-10-08']['TRN_TOTAL'].sum()
c = a['2019-10-09':'2019-10-24']['yhat'].sum()
est = b + c
est

3281183.1203270885

In [ ]:
print(a['2019-09-01':'2019-09-30']['yhat'].sum())
print(a['2019-10-01':'2019-10-31']['yhat'].sum())
print(a['2019-11-01':'2019-11-30']['yhat'].sum())
print(a['2019-12-01':'2019-12-31']['yhat'].sum())
print(a['2019-10-01':'2019-10-31']['yhat'])

In [ ]:
import datetime
import matplotlib.pyplot
teste = a['2019-10-01':'2019-10-31'].copy()
teste = teste.reset_index()
k = teste['ds'].astype(str)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15.5, 4.5)
plt.plot(k,teste[['yhat','y']])
plt.xticks(rotation='vertical')
plt.show()
#print(teste[['yhat','y']])

In [ ]:
print(dataset.set_index('DATA')['2018-01-01':'2018-01-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-02-01':'2018-02-28']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-03-01':'2018-03-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-04-01':'2018-04-30']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-05-01':'2018-05-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-06-01':'2018-06-30']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-07-01':'2018-07-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-08-01':'2018-08-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2018-09-01':'2018-09-30']['TRN_TOTAL'].sum())
print('  ')
print(dataset.set_index('DATA')['2019-01-01':'2019-01-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-02-01':'2019-02-28']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-03-01':'2019-03-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-04-01':'2019-04-30']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-05-01':'2019-05-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-06-01':'2019-06-30']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-07-01':'2019-07-31']['TRN_TOTAL'].sum())
print(dataset.set_index('DATA')['2019-08-01':'2019-08-31']['TRN_TOTAL'].sum())
print(a['2019-09-01':'2019-09-30']['yhat'].sum())